In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
import tensorflow as tf

2023-08-18 10:03:17.759825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv("gas_turbines.csv")
df.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


In [4]:
df.isna().sum()

AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TIT     0
TAT     0
TEY     0
CDP     0
CO      0
NOX     0
dtype: int64

In [5]:
df.drop_duplicates()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.2494,93.227


In [6]:
df.describe()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
count,15039.000000,15039.00000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000
mean,17.764381,1013.19924,79.124174,4.200294,25.419061,1083.798770,545.396183,134.188464,12.102353,1.972499,68.190934
std,7.574323,6.41076,13.793439,0.760197,4.173916,16.527806,7.866803,15.829717,1.103196,2.222206,10.470586
min,0.522300,985.85000,30.344000,2.087400,17.878000,1000.800000,512.450000,100.170000,9.904400,0.000388,27.765000
25%,11.408000,1008.90000,69.750000,3.723900,23.294000,1079.600000,542.170000,127.985000,11.622000,0.858055,61.303500
50%,18.186000,1012.80000,82.266000,4.186200,25.082000,1088.700000,549.890000,133.780000,12.025000,1.390200,66.601000
75%,23.862500,1016.90000,90.043500,4.550900,27.184000,1096.000000,550.060000,140.895000,12.578000,2.160400,73.935500
max,34.929000,1034.20000,100.200000,7.610600,37.402000,1100.800000,550.610000,174.610000,15.081000,44.103000,119.890000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15039 entries, 0 to 15038
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      15039 non-null  float64
 1   AP      15039 non-null  float64
 2   AH      15039 non-null  float64
 3   AFDP    15039 non-null  float64
 4   GTEP    15039 non-null  float64
 5   TIT     15039 non-null  float64
 6   TAT     15039 non-null  float64
 7   TEY     15039 non-null  float64
 8   CDP     15039 non-null  float64
 9   CO      15039 non-null  float64
 10  NOX     15039 non-null  float64
dtypes: float64(11)
memory usage: 1.3 MB


In [8]:
a = StandardScaler()
a.fit(df)
df_s = a.transform(df)

In [9]:
# array = df_s.values
x_arr = df_s[:, (0,1,2,3,4,5,6,8,9,10)]
y = df_s[:, 7]
x_arr,y

(array([[-1.4397781 , -0.82664395,  1.28143632, ..., -1.35733078,
          0.53201195,  1.3878449 ],
        [-1.44960109, -0.74864748,  1.30456402, ..., -1.36367619,
          0.56873344,  1.39300237],
        [-1.43472138, -0.68625031,  1.21908576, ..., -1.36095673,
          0.5529378 ,  1.36358566],
        ...,
        [-1.38626659, -1.07623263,  1.47697056, ..., -1.46792219,
          2.69592467,  2.17006209],
        [-1.42042259, -0.99823616,  1.44159024, ..., -1.42259784,
          1.9246834 ,  2.391165  ],
        [-1.43073409, -0.93583899,  1.33465179, ..., -1.37727349,
          1.35415028,  2.32153907]]),
 array([-1.23117246, -1.22990897, -1.23054071, ..., -1.5160891 ,
        -1.48134317, -1.42827666]))

In [10]:
pd.DataFrame(x_arr).describe()

,0,1,2,3,4,5,6,7,8,9
count,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04,1.503900e+04
mean,4.913654e-16,-1.875504e-14,2.570219e-16,1.511894e-16,-7.559468e-17,-3.260021e-15,1.640405e-15,1.965462e-16,3.779734e-17,4.233302e-16
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00
min,-2.276462e+00,-4.266288e+00,-3.536594e+00,-2.779497e+00,-1.806771e+00,-5.021933e+00,-4.188141e+00,-1.992416e+00,-8.874862e-01,-3.861033e+00
25%,-8.392292e-01,-6.706510e-01,-6.796337e-01,-6.266930e-01,-5.091458e-01,-2.540512e-01,-4.101146e-01,-4.354335e-01,-5.015202e-01,-6.578107e-01
50%,5.566605e-02,-6.227861e-02,2.277844e-01,-1.854065e-02,-8.075681e-02,2.965544e-01,5.712570e-01,-7.011925e-02,-2.620452e-01,-1.518527e-01
75%,8.051309e-01,5.772924e-01,7.916582e-01,4.612196e-01,4.228638e-01,7.382490e-01,5.928675e-01,4.311680e-01,8.455882e-02,5.486567e-01
max,2.266234e+00,3.275970e+00,1.528011e+00,4.486233e+00,2.871006e+00,1.028678e+00,6.627839e-01,2.700105e+00,1.895949e+01,4.937717e+00


In [14]:
type(x_arr), type(y)

(numpy.ndarray, numpy.ndarray)

In [15]:
def create_model():
    model = Sequential()
    
    model.add(Dense(10, input_dim=10, kernel_initializer='uniform', activation='relu'))
    
    model.add(Dense(32,kernel_initializer='uniform', activation='relu'))
    
    model.add(Dense(32,kernel_initializer='uniform', activation='relu'))

    model.add(Dense(16,kernel_initializer='uniform', activation='relu'))

    model.add(Dense(16,kernel_initializer='uniform', activation='relu'))
    
    model.add(Dense(1, kernel_initializer='uniform'))
    
    adam=Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=adam)
    return model

In [16]:
from scikeras.wrappers import KerasRegressor

In [17]:
model = KerasRegressor(build_fn = create_model,verbose = 0)
batch_size = [10,20,40]
epochs = [10,50,100, 150]
param_grid = dict(batch_size = batch_size,epochs = epochs)

# Build and fit the GridSearchCV
grid = RandomizedSearchCV(estimator = model,param_distributions = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(x_arr,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START batch_size=10, epochs=10...................................


/Users/hari/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 1/5; 1/10] END ....batch_size=10, epochs=10;, score=0.996 total time=  25.0s
[CV 2/5; 1/10] START batch_size=10, epochs=10...................................


/Users/hari/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 2/5; 1/10] END ....batch_size=10, epochs=10;, score=0.996 total time=  20.8s
[CV 3/5; 1/10] START batch_size=10, epochs=10...................................


...

/Users/hari/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 4/5; 10/10] END ..batch_size=10, epochs=150;, score=0.997 total time= 7.4min
[CV 5/5; 10/10] START batch_size=10, epochs=150.................................


/Users/hari/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 5/5; 10/10] END ..batch_size=10, epochs=150;, score=0.998 total time= 6.2min


/Users/hari/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


In [18]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.997682332641449, using {'epochs': 150, 'batch_size': 40}
0.9972111835089358,0.0011131659855082292 with: {'epochs': 10, 'batch_size': 10}
0.9974290564879453,0.0009908574867216347 with: {'epochs': 50, 'batch_size': 20}
0.997682332641449,0.000774452414141169 with: {'epochs': 150, 'batch_size': 40}
0.789365958677253,0.4155151531818856 with: {'epochs': 50, 'batch_size': 10}
0.7788141900814514,0.4370585176494598 with: {'epochs': 10, 'batch_size': 20}
0.9976772545175738,0.0009055437988719426 with: {'epochs': 100, 'batch_size': 40}
0.9973686579044332,0.0007989724804636089 with: {'epochs': 10, 'batch_size': 40}
0.997523546050353,0.0009317702729376102 with: {'epochs': 50, 'batch_size': 40}
0.9973950407220663,0.0006646730615778958 with: {'epochs': 150, 'batch_size': 20}
0.7782217386392588,0.4380459825874554 with: {'epochs': 150, 'batch_size': 10}
